In [2]:
cd "D:/repo/CS420_Lab02_Connect-4"

D:\repo\CS420_Lab02_Connect-4


In [3]:
from pandas import read_csv

data_raw = read_csv("./data/connect-4.csv", header=0)
print(data_raw)

      a1 a2 a3 a4 a5 a6 b1 b2 b3 b4  ... f4 f5 f6 g1 g2 g3 g4 g5 g6 class
0      b  b  b  b  b  b  b  b  b  b  ...  b  b  b  b  b  b  b  b  b   win
1      b  b  b  b  b  b  b  b  b  b  ...  b  b  b  b  b  b  b  b  b   win
2      b  b  b  b  b  b  o  b  b  b  ...  b  b  b  b  b  b  b  b  b   win
3      b  b  b  b  b  b  b  b  b  b  ...  b  b  b  b  b  b  b  b  b   win
4      o  b  b  b  b  b  b  b  b  b  ...  b  b  b  b  b  b  b  b  b   win
...   .. .. .. .. .. .. .. .. .. ..  ... .. .. .. .. .. .. .. .. ..   ...
67552  x  x  b  b  b  b  o  x  o  b  ...  b  b  b  o  o  x  b  b  b  loss
67553  x  x  b  b  b  b  o  b  b  b  ...  b  b  b  o  x  o  o  x  b  draw
67554  x  x  b  b  b  b  o  o  b  b  ...  b  b  b  o  x  x  o  b  b  loss
67555  x  o  b  b  b  b  o  b  b  b  ...  b  b  b  o  x  o  x  x  b  draw
67556  x  o  o  o  x  b  o  b  b  b  ...  b  b  b  x  b  b  b  b  b  draw

[67557 rows x 43 columns]


In [4]:
data_feature = data_raw.iloc[:, :-1]
data_label = data_raw.iloc[:, -1:]

# feature_name = data_feature.columns
# class_name = 

x = data_feature.to_numpy()
y = data_label.to_numpy()

In [5]:
for i in range( 0, len( x[:] ) ):
    for j in range( 0, len( x[i][:] ) ):
        if x[i][j] == 'b':
            x[i][j] = -1
        elif x[i][j] == 'x':
            x[i][j] = 0
        elif x[i][j] == 'o':
            x[i][j] = 1
        else:
            print(x[i][j])
            raise Exception("Invalid Value")

In [6]:
print(x)
print(y)

[[-1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 ...
 [0 0 -1 ... 1 -1 -1]
 [0 1 -1 ... 0 0 -1]
 [0 1 1 ... -1 -1 -1]]
[['win']
 ['win']
 ['win']
 ...
 ['loss']
 ['draw']
 ['draw']]


In [7]:
from sklearn.model_selection import StratifiedShuffleSplit

datasets = []

train_propotions = [0.4, 0.6, 0.8, 0.9]
for prop in train_propotions:
    splitter = StratifiedShuffleSplit(n_splits=1, train_size=prop)
    
    for train_idx, test_idx in splitter.split(x, y):
        feature_train, label_train = x[train_idx], y[train_idx]    
        feature_test, label_test = x[test_idx], y[test_idx]
             
        datasets.append(
            {"feature_train" : feature_train,
             "label_train" : label_train,
             "feature_test" : feature_test,
             "label_test" : label_test})

In [ ]:
for dataset in datasets:
    print( len(dataset["feature_train"]) / ( len(dataset["feature_train"]) + len(dataset["feature_test"]) ) )

In [10]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from graphviz import Source

dec_trees = []
for dataset in datasets:
    dec_tree = DecisionTreeClassifier(criterion='entropy')
    
    dec_tree = dec_tree.fit(dataset["feature_train"], dataset["label_train"])
    
    dec_trees.append(dec_tree)
    Source(export_graphviz(dec_tree, filled=True))

In [ ]:
graph = Source(export_graphviz(dec_trees[0], filled=True, out_file=None))
graph.format = 'png'
graph.render('dtree_render',view=True)

In [9]:
for dec_tree in dec_trees:
    accus = []
    for dataset in datasets:
        accus.append(dec_tree.score(dataset["feature_test"], dataset["label_test"]))
        
    for accu in accus:
        print(accu)
    print(f"Average: {sum(accus) / len(accus)}")
    print()

0.7261625755519922
0.8365466454501721
0.8332593250444049
0.8352575488454707
Average: 0.80780652372301

0.8979153817688418
0.7442548939792029
0.8968324452338662
0.8922439313203079
Average: 0.8578116630755546

0.9508819538670285
0.9497835177441438
0.7507400828892836
0.9498223801065719
Average: 0.900306983651757

0.9764154434439374
0.9755763608777708
0.9750592066311426
0.7643576080521018
Average: 0.9228521547512382



In [ ]:
from numpy import concatenate
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay
)

for dec_tree, dataset in zip(dec_trees, datasets):
    pred = dec_tree.predict(dataset["feature_test"])
    ground_truth = concatenate(dataset["label_test"])
    
    print( classification_report(ground_truth, pred) )
    
    conf_matrix = confusion_matrix(ground_truth, pred)
    ConfusionMatrixDisplay.from_predictions(ground_truth, pred)

In [ ]:
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.metrics import accuracy_score
from graphviz import Source
from numpy import concatenate

dataset = datasets[2]

max_depths = (None, 2, 3, 4, 5, 6, 7)
for max_depth in max_depths:
    dec_tree = DecisionTreeClassifier(criterion='entropy', max_depth=max_depth)
    dec_tree = dec_tree.fit(dataset["feature_train"], dataset["label_train"])
    
    Source(export_graphviz(dec_tree, filled=True))
    
    pred = dec_tree.predict(dataset["feature_test"])
    ground_truth = concatenate(dataset["label_test"])
    
    print( accuracy_score(ground_truth, pred) )
    